# Flaky Tests

Flaky refers to tests which can be run multiple times with the same input and conditions and yield different results. An example below shows a random number function that will output a number from zero to ***n* (inclusive)** but a test written for it that assumes zero to ***n* (exclusive)**.

This will fail an average of one in every *n* times, but a test suite is likely to run it just once. And statistically, it is likely to pass that once.

In [ ]:
from random import randint

def f(x):
    random = randint(0, x)
    # print selection so we can see it
    print(f"f({x}) = {random}")
    return random

def test():
    n = 10
    assert(f(n) < n)

In [ ]:
test()

f(10) = 2


In [ ]:
for _ in range(15):
    test()
# run this cell until you get an Error

f(10) = 2
f(10) = 7
f(10) = 2
f(10) = 6
f(10) = 4
f(10) = 2
f(10) = 3
f(10) = 3
f(10) = 4
f(10) = 4
f(10) = 7
f(10) = 10


AssertionError: ignored

# Compound Instructions

What many would regard as a single line of code or a single operation is often several to a CPU. Here, some Python code emulates some basic CPU instructions to demonstrate a hypothetical process for the sequence:

```python
b = 1
c = 2
a = b + c
print(a)
```

**Note** all the places where incorrect input or memory state can cause this to crash, as these are all the tasks that would require large amounts of monitoring and error handling and memory management code.

In [ ]:
store = [None, None, None, None] # memory storage
lookup = {} # lookup table for memory locations
A, B, C = ('a', 'b', 'c') # strings for variable name literals

# LOOKUP finds the entry with the name `variable` in the lookup table
# and crashes if it doesn't exist
def LOOKUP(variable):
    return lookup[variable]


# GET retrieves the value at the location `address` in memory, regardless
# of whether it is the type you expect or interpreted correctly or
# malformed, and crashes if that memory location is inaccessible
def GET(address):
    return store[address]


# SET sets the value of the location `address` in memory, regardless of
# whether it has been entered in the lookup table so it can be ever found again
# and will malform other parts of memory if this new value requires more bits
# than the old one
def SET(address, value):
    store[address] = value


# CREATE finds the next empty location in memory, registers a new entry named
# `variable` in the lookup table that references this location,
# and crashes if there are no empty locations of suitable size
def CREATE(variable):
    free_location = None
    for location, contents in enumerate(store):
        if contents is None:
            free_location = location
            break
    lookup[variable] = free_location


# SUM adds two raw values together, and crashes if they are non-numeric or
# non-matching types
def SUM(a, b):
    return a + b


# SHOW displays a value on screen, and crashes if it cannot be coerced into
# a string representation
def SHOW(var):
    print(var)

In [ ]:
# b = 1
CREATE(B)
SET(LOOKUP(B), 1)

# c = 2
CREATE(C)
SET(LOOKUP(C), 2)

# a = b + c
CREATE(A)
SET(LOOKUP(A), SUM(GET(LOOKUP(B)), GET(LOOKUP(C))))

# print(a)
SHOW(GET(LOOKUP(A)))

3


# Mercurial Cores in Distributed Computing

Multiple axes of randomness make detection or observation of fail-silent errors difficult; a task has to be allocated to the correct core on the correct machine in the correct configuration with the correct instruction, after a possible catalyst that is age and level of use, for the issue to occur.

Below is an example of task distribution within a data centre where a random machine is randomly error-prone.

In [ ]:
def random_chance():
    return randint(0, 10) == 10

# A Task is a set of the same instructions to perform for multiple inputs
# that can produce parallelizable subtasks from itself so that work
# can be done on multiple machines at once. Afterwards, it collates results.
class Task:
    def __init__(self, function, input):
        self.function = function
        self.input = input
        self.num_subtasks = len(input)
        self.results = [None] * self.num_subtasks
        self.num_complete_subtasks = 0

    def get_parellelizeable_subtask(self, n):
        return (self.function, self.input[n])

    def set_subtask_result(self, n, result):
        self.results[n] = result
        self.num_complete_subtasks += 1
        if self.num_complete_subtasks == self.num_subtasks:
            self.alert_all_subtasks_complete()

    def alert_all_subtasks_complete(self):
        print("Results: ", self.results)

# A Machine does the actual computation.
class Machine:
    def __init__(self):
        self.is_mercurial = False

    def make_mercurial(self):
        self.is_mercurial = True

    def compute(self, function, input):
        if self.is_mercurial and random_chance():
            return input # the wrong answer
        return function(input)

# A DataCentre is a collection of machines that get delegated work
# on some availability or resource or random basis.
class DataCentre:
    def __init__(self, num_machines = 3):
        self.num_machines = num_machines
        self.machines = [Machine()] * num_machines
        self.machines[num_machines - 1].make_mercurial() # one bad machine
        self.current_machine = 0

    def get_random_machine(self):
        n = randint(0, self.num_machines - 1)
        return self.machines[n]

    def distribute(self, task):
        for n in range(task.num_subtasks):
            machine = self.get_random_machine()
            subtask_function, subtask_input = task.get_parellelizeable_subtask(n)
            result = machine.compute(subtask_function, subtask_input)
            task.set_subtask_result(n, result)

In [ ]:
def example_function(x):
    return x * 2

example_input = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
print("Expected:", [example_function(x) for x in example_input])

task = Task(example_function, example_input)
DataCentre().distribute(task)

Expected: [0, 2, 4, 6, 8, 10, 12, 14, 16, 18]
Results:  [0, 2, 4, 6, 8, 5, 12, 14, 16, 18]


# Extra Credit: Spectre

Branch prediction leads to potential execution of instructions that should never be run. Instructions run this way were not subject to the same memory access protections because if the predicted branch didn't end up being correct, the results would be thrown away. So, of course, people conceived of a way to get information out of functionality that could not return results: observe the *time* it took to do certain checks.

In [ ]:
from time import time

protected_memory = [1, 0, 1, 1, 0, 1, 0, 0]
num_bits = len(protected_memory)
allowed_access = False

def pipeline(condition, operation, arg):
    result = operation(arg)
    if condition:
        return result
    else:
        return None

def read_bit(n):
    return protected_memory[n]

In [ ]:
# attempting to access bits will yield blank results
normal = []

for n in range(num_bits):
    bit = pipeline(allowed_access, read_bit, n)
    normal.append(bit)

print("Memory:", protected_memory)
print("Read:  ", normal)

Memory: [1, 0, 1, 1, 0, 1, 0, 0]
Read:   [None, None, None, None, None, None, None, None]


In [ ]:
threshold = 0.001

def long_task(): # in real Spectre, the slow operation was to access L4 cache
    n = 1
    for i in range(100):
        for j in range(100):
            for k in range(100):
                n *= (i * j * k)
    return

def short_task(): # the short operation was a no-op
    return

def read_bit_spectre(n):
    bit = protected_memory[n]
    if bit == 1:
        long_task()
    else:
        short_task()

In [ ]:
# read inaccessible memory without ever returning a result
spectre = []

for n in range(num_bits):
    start = time()
    # run the operation but throw away result
    pipeline(allowed_access, read_bit_spectre, n)
    end = time()
    elapsed = end - start
    # instead observe time taken
    bit = 1 if elapsed > threshold else 0
    spectre.append(bit)

print("Memory: ", protected_memory)
print("Spectre:", spectre)

Memory:  [1, 0, 1, 1, 0, 1, 0, 0]
Spectre: [1, 0, 1, 1, 0, 1, 0, 0]


In [ ]:
from time import sleep, time

def read_bit(n):
    bit = protected_memory[n]
    if bit == 1:
        # do something slow
        sleep(0.1)
        # (in irl Spectre, this was "access L4 cache")
        return
    else:
        # do something fast
        return

# read inaccessible memory without ever returning a result
def read_protected_memory():
    result = []
    for n in range(len(protected_memory)):
        start = time()
        # run the operation but throw away result
        instruction(read_bit, n)
        end = time()
        elapsed = end - start
        # instead observe time taken
        bit = 1 if elapsed >= 0.01 else 0
        result.append(bit)
    return result


In [ ]:
print("Result:", read_protected_memory())

Result: [1, 0, 1, 1, 0, 1, 0, 0]
